<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/deep%20learning/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Guia Prático de Fine-Tuning: Llama 3 8B com QLoRA

Olá! Este notebook é seu guia passo a passo para o mundo do fine-tuning de LLMs. Como estudante de Engenharia de Computação, é crucial entender não apenas *como* fazer, mas *por que* as técnicas funcionam.



As Estratégias de Fine-Tuning

1.  **Full Fine-Tuning**:
    * **O que é?** Ajustar *todos* os bilhões de pesos do modelo.
    * **Problema:** Requer uma quantidade massiva de VRAM (memória de GPU), sendo inviável para modelos de 8B em hardware comum ou no Colab. Um modelo de 8B com precisão total (32-bit) precisaria de `8B * 4 bytes = 32 GB` de VRAM só para ser carregado, sem contar a memória extra para o treinamento.

2.  **PEFT (Parameter-Efficient Fine-Tuning)**:
    * **O que é?** Uma família de técnicas que congela os pesos originais do LLM (que são 99.9% do total) e treina apenas um número minúsculo de novos parâmetros "adaptadores".
    * **Vantagem:** Redução drástica no uso de memória e aceleração do treino.

3.  **LoRA (Low-Rank Adaptation)**:
    * **A Estrela do PEFT.** A ideia é que a "atualização" dos pesos para uma nova tarefa pode ser representada por matrizes de baixo posto (muito menores). Em vez de modificar uma matriz de peso gigante `W`, adicionamos o resultado de duas matrizes pequenas, `A` e `B`, que são treináveis (`W' = W + B*A`).

4.  **QLoRA (Quantized LoRA)**:
    * **A Magia que nos permite rodar no Colab.** É uma otimização do LoRA que faz duas coisas geniais:
        1.  **Quantização:** Carrega o modelo principal (Llama 3 8B) com precisão reduzida (4-bit em vez de 16-bit), cortando o uso de memória em 4x.
        2.  **Adaptação LoRA:** Treina os adaptadores LoRA normalmente sobre esse modelo quantizado.

**Nosso objetivo hoje:** Vamos ensinar o `Meta Llama 3 8B Instruct` a responder a um comando em linguagem natural com um output em formato JSON bem estruturado, uma tarefa muito útil em engenharia de software.

## Parte 1

In [1]:
!pip install -q -U transformers accelerate bitsandbytes peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 51.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from google.colab import userdata

# Retrieve the HF_TOKEN from Colab secrets and login
hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)

In [28]:
# Para este exemplo, não vamos usar um dataset externo.
# Vamos criar um pequeno dataset na mão para ensinar o modelo a estruturar saídas em JSON.

from datasets import Dataset

# Nosso objetivo é transformar um texto não estruturado sobre um usuário em um JSON.
instructions = [
    "Extraia o nome, a idade e a cidade de: 'O usuário João Silva tem 28 anos e mora em São Paulo.'",
    "Transforme em JSON os dados de: 'Ana Souza, 35 anos, residente do Rio de Janeiro.'",
    "Formate as seguintes informações: 'Carlos Pereira, de Curitiba, tem 42 anos.'",
    "Converta para JSON: 'Maria Fernandes, 30 anos, vive em Belo Horizonte.'",
    "Gere um JSON para: 'Pedro Costa, 22 anos, natural de Porto Alegre.'",
    "Estruture em JSON os dados de: 'Luiza Mendes, 45 anos, moradora de Fortaleza.'",
    "Extraia e formate em JSON: 'Fernando Rocha, 50 anos, reside em Recife.'",
    "Converta os detalhes para JSON: 'Patrícia Alves, 29 anos, de Salvador.'",
    "Crie um JSON para: 'Gustavo Ribeiro, 38 anos, morador de Brasília.'",
    "Formate os dados em JSON: 'Carolina Dias, 27 anos, vive em Curitiba.'"

]

outputs = [
    '{"nome": "João Silva", "idade": 28, "cidade": "São Paulo"}',
    '{"nome": "Ana Souza", "idade": 35, "cidade": "Rio de Janeiro"}',
    '{"nome": "Carlos Pereira", "idade": 42, "cidade": "Curitiba"}',
    '{"nome": "Maria Fernandes", "idade": 30, "cidade": "Belo Horizonte"}',
    '{"nome": "Pedro Costa", "idade": 22, "cidade": "Porto Alegre"}',
    '{"nome": "Luiza Mendes", "idade": 45, "cidade": "Fortaleza"}',
    '{"nome": "Fernando Rocha", "idade": 50, "cidade": "Recife"}',
    '{"nome": "Patrícia Alves", "idade": 29, "cidade": "Salvador"}',
    '{"nome": "Gustavo Ribeiro", "idade": 38, "cidade": "Brasília"}',
    '{"nome": "Carolina Dias", "idade": 27, "cidade": "Curitiba"}'
]

# O formato do prompt é crucial para modelos "instruct".
formatted_data = []
for instruction, output in zip(instructions, outputs):
    # Formato do Gemma: <start_of_turn> e <end_of_turn>
    text = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{output}<end_of_turn>"
    formatted_data.append({"text": text})

# Criando um objeto Dataset da Hugging Face
dataset = Dataset.from_dict({"text": [item["text"] for item in formatted_data]})

print("Exemplo de um item do dataset formatado:")
print(dataset[0]['text'])

Exemplo de um item do dataset formatado:
<start_of_turn>user
Extraia o nome, a idade e a cidade de: 'O usuário João Silva tem 28 anos e mora em São Paulo.'<end_of_turn>
<start_of_turn>model
{"nome": "João Silva", "idade": 28, "cidade": "São Paulo"}<end_of_turn>


In [37]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Carregando o modelo base
model_id = "google/gemma-2b"

# Configuração de quantização (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype="auto"
)

# Carregando o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


# Ativa otimizações para o treino com quantização
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# Configuração dos adaptadores LoRA
lora_config = LoraConfig(
    r=8,  # "rank" da decomposição. Um valor maior treina mais parâmetros, mas pode levar a overfitting. 16 é um bom começo.
    lora_alpha=16, # Parâmetro de escala. A regra geral é que seja 2 * r.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Common target modules for decoder models
    lora_dropout=0.05, # Dropout para regularização
    bias="none",
    task_type="CAUSAL_LM" # Tarefa de modelagem de linguagem causal
)

# Aplica o wrapper PEFT no nosso modelo
peft_model = get_peft_model(model, lora_config)

# Imprime o número de parâmetros treináveis para vermos a mágica do PEFT
peft_model.print_trainable_parameters()
# Você verá que o número de parâmetros treináveis é < 1% do total!

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 9,805,824 || all params: 2,515,978,240 || trainable%: 0.3897


In [38]:
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

# Seus Argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./google/gemma-2b-json-finetune",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    max_grad_norm=0.3,
    num_train_epochs=50,
    logging_steps=10,
    bf16=True,
    push_to_hub=False,
    report_to="wandb"
)

# 3. Passe o 'config' para o SFTTrainer e remova os argumentos antigos
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    args=training_args,
)

# Inicia o treinamento
trainer.train()

Adding EOS to train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,4.246200
20,4.243900
30,4.207800
40,4.245200
50,4.210900
60,4.260900
70,4.267500


KeyboardInterrupt: 

## Parte 2

Parabéns, você treinou um adaptador LoRA! No entanto, no estado atual, você tem duas partes: o modelo base gigante (Llama 3 8B) e seu pequeno adaptador LoRA. Para fazer uma inferência, você precisa carregar ambos.

A **mesclagem** é o processo de "fundir" os pesos do seu adaptador LoRA de volta aos pesos do modelo base. O resultado é um **único modelo autônomo** que já contém a sua especialização.

### Por que mesclar?

1.  **Simplificação de Deploy:** Em vez de gerenciar o modelo base + o adaptador, você distribui um único modelo. É muito mais simples para colocar em produção.
2.  **Performance de Inferência:** A inferência pode ser ligeiramente mais rápida, pois o modelo não precisa mais combinar dinamicamente os pesos do LoRA com os pesos base a cada passada. O cálculo `W' = W + B*A` já foi feito e "assado" no modelo.
3.  **Compartilhamento:** Permite que você compartilhe seu modelo fine-tuned completo na Hugging Face como um novo modelo, e não apenas como um adaptador.

Vamos ver como fazer isso na prática.

In [31]:
# Célula 11: Carregando o Modelo Base e o Adaptador para Mesclagem
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# --- Carregando o Modelo Base ---
# Desta vez, vamos carregar o modelo em precisão reduzida (4-bit)
# para a mesclagem, caso a versão de 16-bit exija muita memória.

model_id = "google/gemma-2b"

# Configuração de quantização (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # Use the 4-bit quantization config
    device_map="auto", # Explicitly set device_map to auto
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# --- Carregando o Adaptador PEFT ---
# Apontamos para o diretório onde o `trainer` salvou nosso adaptador LoRA.
adapter_path = "./google/gemma-2b-json-finetune/checkpoint-100"
peft_model = PeftModel.from_pretrained(base_model, adapter_path)

print("Modelo base e adaptador carregados com sucesso.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo base e adaptador carregados com sucesso.


In [32]:
# Célula 12: Executando a Mesclagem e Testando o Modelo Final

# Este é o comando mágico que funde os pesos!
# Ele "descarrega" o wrapper PEFT e retorna um modelo transformers padrão.
merged_model = peft_model.merge_and_unload()

print("Mesclagem concluída!")

# --- Teste do Modelo Mesclado ---
# Agora, podemos usar este `merged_model` como qualquer outro modelo da Hugging Face.
# Note que não precisamos mais do objeto `peft_model` aqui.

from transformers import pipeline

# O mesmo prompt de teste que usamos antes, formatado manualmente
# according to the structure defined in cell DiPtigwsBGly.
test_instruction = "Extraia as informações de 'Mariana Lima, 25 anos, de Salvador.' em formato JSON."
# Use the Gemma prompt format
test_prompt = f"<start_of_turn>user\n{test_instruction}<end_of_turn>\n<start_of_turn>model\n"


# Criamos um pipeline com o nosso NOVO modelo mesclado
merged_pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer)

# Executamos a inferência
# Set max_new_tokens to a reasonable value to avoid infinite generation.
merged_output = merged_pipe(test_prompt, max_new_tokens=50, do_sample=False)

print("\n--- Resposta do Modelo Mesclado e Autônomo ---")
print(merged_output[0]['generated_text'])

# O resultado deve ser idêntico ao do modelo com o adaptador PEFT,
# provando que a mesclagem foi um sucesso!

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Device set to use cuda:0


Mesclagem concluída!

--- Resposta do Modelo Mesclado e Autônomo ---
<start_of_turn>user
Extraia as informações de 'Mariana Lima, 25 anos, de Salvador.' em formato JSON.<end_of_turn>
<start_of_turn>model
Extraia as informações de 'Mariana Lima, 25 anos, de Salvador.' em formato JSON. €)
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows
SneakyThrows


In [36]:
# Célula 13: Salvando o Modelo Mesclado Completo para o Disco

# Agora você tem um modelo completo. Vamos salvá-lo.
# Este diretório conterá todos os arquivos necessários para carregar o modelo
# sem precisar do código do fine-tuning ou dos adaptadores.

output_merged_dir = "./google/gemma-2b-json-finetune" # Updated output directory

merged_model.save_pretrained(output_merged_dir)
tokenizer.save_pretrained(output_merged_dir)

print(f"Modelo mesclado completo salvo em: {output_merged_dir}")
# A partir daqui, você poderia carregar este modelo com um simples:
# AutoModelForCausalLM.from_pretrained("./google/gemma-2b-json-finetune")

Modelo mesclado completo salvo em: ./google/gemma-2b-json-finetune


## Saiba Mais

E se você tivesse treinado **vários adaptadores** LoRA?
* Um para gerar JSON (o que fizemos).
* Outro para ser um especialista em programação Python.
* Um terceiro para escrever de forma criativa.

Você pode mesclá-los para criar um "super-modelo" que faz tudo isso. A função `.merge_and_unload()` não é ideal para isso. A ferramenta padrão da comunidade é a **`mergekit`**.

`mergekit` funciona com um arquivo de configuração YAML onde você especifica os modelos e a estratégia de mesclagem.
